In [1]:
#Import drive
from google.colab import drive
#Mount Google Drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Colab Notebooks/Sebastian

/content/drive/MyDrive/Colab Notebooks/Sebastian


Stuff for commiting to Github below

In [ ]:
ROOT = '/content/drive"'

In [ ]:
from os.path import join  

# path to your project on Google Drive
MY_GOOGLE_DRIVE_PATH = 'drive/MyDrive/Colab Notebooks/Sebastian' 
# replace with your Github username 
GIT_USERNAME = "ahhuang007" 
# definitely replace with your
GIT_TOKEN = "new github token here"  
# Replace with your github repository in this case we want 
# to clone deep-learning-v2-pytorch repository
GIT_REPOSITORY = "Sebastian" 

PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH)

# It's good to print out the value if you are not sure 
print("PROJECT_PATH: ", PROJECT_PATH)   

# In case we haven't created the folder already; we will create a folder in the project path     

#GIT_PATH = "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" this return 400 Bad Request for me
#GIT_PATH = "https://" + GIT_TOKEN + "@github.com/" + GIT_USERNAME + "/" + GIT_REPOSITORY + ".git"
#print("GIT_PATH: ", GIT_PATH)

PROJECT_PATH:  /content/drive"/drive/MyDrive/Colab Notebooks/Sebastian


In [ ]:
!git add .

The file will have its original line endings in your working directory.


In [ ]:
!git config --global user.email "ahhuang007@gmail.com"
!git config --global user.name "ahhuang007"

In [ ]:
!git commit -m "changed some conditions for ending episodes, still not performing better though"

[main 7a510df] changed some conditions for ending episodes, still not performing better though
 7 files changed, 3 insertions(+), 6 deletions(-)
 rewrite colab.ipynb (91%)
 rewrite gym-seb/gym_seb/envs/__pycache__/seb_env.cpython-37.pyc (69%)
 rewrite random_model_ppo.zip (78%)
 rewrite real_model_ppo.zip (89%)


Had to reset the remote origin URL here, removed the code since it contained the personal authorization token.

--------

Command for pulling from github

In [3]:
!git pull origin

remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 11 (delta 6), reused 11 (delta 6), pack-reused 0
Unpacking objects: 100% (11/11), done.
From https://github.com/ahhuang007/Sebastian
   f6e7db8..421e480  main       -> origin/main
Updating f6e7db8..421e480
error: Your local changes to the following files would be overwritten by merge:
	gym-seb/gym_seb/envs/__pycache__/seb_env.cpython-37.pyc
	gym-seb/gym_seb/envs/seb_env.py
Please commit your changes or stash them before you merge.
Aborting


In [ ]:
!git push origin

Counting objects: 14, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (14/14), done.
Writing objects: 100% (14/14), 167.27 KiB | 6.97 MiB/s, done.
Total 14 (delta 5), reused 0 (delta 0)
remote: Resolving deltas: 100% (5/5), completed with 5 local objects.
To https://github.com/ahhuang007/Sebastian.git
   5181969..7a510df  main -> main


Getting OpenAI Baselines

In [ ]:
!git clone https://github.com/openai/baselines.git

Cloning into 'baselines'...
remote: Enumerating objects: 3627, done.
remote: Total 3627 (delta 0), reused 0 (delta 0), pack-reused 3627
Receiving objects: 100% (3627/3627), 6.46 MiB | 8.34 MiB/s, done.
Resolving deltas: 100% (2429/2429), done.
Checking out files: 100% (199/199), done.


Baselines doesn't come installed, tragic

In [ ]:
cd baselines

/content/drive/MyDrive/Colab Notebooks/Sebastian/baselines


In [ ]:
!pip install -e .

Installing stable_baselines since it might be better

In [ ]:
%cd ..

/content/drive/MyDrive/Colab Notebooks


In [4]:
!pip install stable-baselines3[extra]

     |████████████████████████████████| 174 kB 5.2 MB/s 


Installing my custom environment

In [5]:
cd Sebastian

[Errno 2] No such file or directory: 'Sebastian'
/content/drive/MyDrive/Colab Notebooks/Sebastian


In [6]:
%cd gym-seb

/content/drive/My Drive/Colab Notebooks/Sebastian/gym-seb


In [7]:
!pip install -e .

Obtaining file:///content/drive/My%20Drive/Colab%20Notebooks/Sebastian/gym-seb
     |████████████████████████████████| 1.5 MB 5.4 MB/s 
     |████████████████████████████████| 89.3 MB 63 kB/s 
  Created wheel for gym: filename=gym-0.21.0-py3-none-any.whl size=1616823 sha256=610f3dc6a5f0fd5320801b42a67f359cfb476cc8591b8098e256934bf003e9fb
  Stored in directory: /root/.cache/pip/wheels/76/ee/9c/36bfe3e079df99acf5ae57f4e3464ff2771b34447d6d2f2148
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.17.3
    Uninstalling gym-0.17.3:
      Successfully uninstalled gym-0.17.3
  Running setup.py develop for gym-seb
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
stable-baselines3 1.3.0 requires gym<0.20,>=0.17, but you have gym 0.21.0 which is incompatible.


Restart runtime here

In [1]:
%cd drive/My Drive/Colab Notebooks/Sebastian

/content/drive/My Drive/Colab Notebooks/Sebastian


In [2]:
from stable_baselines3 import DDPG, PPO
import gym

In [3]:
import gym_seb
env = gym.make('seb-v0', max_timesteps = 10000, use_gui = False)

In [4]:
from stable_baselines3.common.env_checker import check_env

check_env(env, warn=True)

resetting environment


In [5]:
from stable_baselines3.ddpg.policies import MlpPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

'''
model finished running when "doneness" was achievable - do i need to do this?
I think the model will keep learning until it is done - if it is done but 
has not reached the number of total timesteps, it will reset and learn again
until it has surpassed the total_timesteps and is done again
'''

model = PPO('MlpPolicy', env, verbose = 1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [6]:
from stable_baselines3.common.evaluation import evaluate_policy

#performance with random model
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
mean_reward=500.10 +/- 0.00011390291739354346


In [7]:
model.save("random_model_ppo")

In [8]:
env.reset()
#set total_timesteps equal to n_eval_episodes * max_timesteps
model = model.learn(total_timesteps = 200000, n_eval_episodes = 10, log_interval = 10000)

resetting environment
resetting environment
robot has flipped over at timestep 207
resetting environment
robot has flipped over at timestep 363
resetting environment
robot has flipped over at timestep 282
resetting environment
robot has flipped over at timestep 731
resetting environment
robot has flipped over at timestep 128
resetting environment
robot has flipped over at timestep 95
resetting environment
robot has flipped over at timestep 548
resetting environment
robot has flipped over at timestep 2328
resetting environment
robot has flipped over at timestep 427
resetting environment
robot has flipped over at timestep 1669
resetting environment
robot has flipped over at timestep 148
resetting environment
robot has flipped over at timestep 3294
resetting environment
robot has flipped over at timestep 295
resetting environment
robot has flipped over at timestep 698
resetting environment
robot has flipped over at timestep 319
resetting environment
robot has flipped over at timestep 705


In [9]:
model.save("real_model_ppo")

In [10]:

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10, deterministic=True)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.7/dist-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
hasn't gone far enough
resetting environment
mean_reward=469.75 +/- 0.2627098939175385


So, yeah, my algorithm's improving but it still needs to improve a lot more. It would appear that PPO is better than DDPG, so I think I'll be working with PPO from here on out.

Possible methods of improvement: Expanding observation space to include joint angles/velocities, contact forces, rotation matrix, increase reward based on velocity and magnitude of action taken, 